## APS2 - TRANSFERÊNCIA DE CALOR E MECÂNICA DOS SÓLIDOS
Grupo: Fernando Fincatti, Gabriela Moreno, Lais Nascimento, Theo Barbara;

---

### Importando as bibliotecas
---

In [2]:
import numpy as np
from math import *
import matplotlib.pyplot as plt

### Declarando varíaveis
---

In [3]:
n = 2              #nº do grupo
k = 1              #m^2/s
alpha = 1          #m/s
T = 3              #s
Q = 100            #kg/ms
lx = 30            #m
ly = 20            #m
a = int(n/1.4)     #xxx
b = int(60/(n+5))  #xxx
tempo = 10*T       #s
delta_t = 1e-3     #s
delta_x = 1        #m
delta_y = delta_x  #m

In [4]:
#teste de convergência
(delta_t/delta_x**2) < (1/4*k)

True

In [5]:
#número de nós em x
dimensao_x = int((lx/delta_x)) + 1

#número de nós em y
dimensao_y = int((ly/delta_y)) + 1

#steps no tempo
dimensao_z = int(tempo/delta_t)

print("Dimensão x: {0}\nDimensão y: {1}\nDimensão z: {2}".format(dimensao_x,dimensao_y,dimensao_z))

Dimensão x: 31
Dimensão y: 21
Dimensão z: 30000


## Funções de auxílio
---

In [7]:
def equacao_concentracao(x, c_mais_i_ant, c_menos_i_ant, c_mais_j_ant, c_menos_j_ant, c_ij_ant):
    
    #calcula o primeiro termo
    primeiro_termo = -alpha*(c_mais_i_ant - c_menos_i_ant)/(2*delta_x)
    
    #calcula o segundo termo
    segundo_termo = -alpha*sin(pi*x/5)*(c_mais_j_ant - c_menos_j_ant)/(2*delta_y)
    
    #calcula o terceiro termo
    terceiro_termo = k*(c_mais_i_ant - 2*c_ij_ant + c_menos_i_ant)/(delta_x**2)
    
    #calcula o quarto termo
    quarto_termo = k*(c_mais_j_ant - 2*c_ij_ant + c_menos_j_ant)/(delta_y**2)
    
    #calcula o quinto termo
    quinto_termo = Q/(delta_x*delta_y)
    
    #calcula a resposta final
    resposta = delta_t*(primeiro_termo + segundo_termo + terceiro_termo + quarto_termo + quinto_termo) + c_ij_ant
    
    return resposta

## Inicializando a placa
---

In [8]:
#inicializa as condições do rio
rio = np.zeros((dimensao_x, dimensao_y, dimensao_z))

In [12]:
rio;

## Calculando a temperatura
---

In [ ]:
tempo_atual = 1  #tempo
i = 1            #linhas
j = 0            #colunas

while(tempo_atual < dimensao_z):
    
    i = 1 #reinicia as colunas
    
    while(i < dimensao_x-1):
        
        j = 1 #reinicia as linhas
        
        while(j < dimensao_y-1):
            # Para a posição (a,b) para tempos menores de 3 segundos tem derramamento Q. !=0
            if(i==a and j==b and tempo_atual<3):
                rio[i, j, tempo_atual]=equacao_concentracao(i, rio[i+1, j, tempo-1], rio[i-1, j, tempo-1], rio[i, j+1, tempo-1], rio[i, j-1, tempo-1], rio[i, j, tempo-1])
            # Se não obdece essas condições tem que desconsiderar o quinto termo que colocamos na função?
            else:
                
            
                
                
                
            j += 1
        i += 1
    tempo_atual += 1

### Condição de contorno

In [ ]:
# Preenche a primeira linha igual a linha de baixo
j = 0
while(j < dimensao_y-1):
    rio[0, j, tempo_atual] = rio[1, j, tempo_atual]
    j++

# Preenche a primeira coluna igual a segunda coluna
i=0
while(i < dimensao_x-1):
    rio[i, 0, tempo_atual] = rio[i, 1, tempo_atual]
    i++
    
# Preenche a ultima linha igual a penultima
j = 0
while(j < dimensao_y-1):
    rio[dimensao_x-1, j, tempo_atual] = rio[dimensao_x-2, j, tempo_atual]
    j++
    
# Preenche a ultima coluna igual a penultima
i=0
while(i < dimensao_x-1):
    rio[i, dimensao_y-1, tempo_atual] = rio[i, dimensao_y-2, tempo_atual]
    i++